<a href="https://colab.research.google.com/github/maurottito/Deploy_cred_prend/blob/main/py_cred_pren_AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pyreadstat

     |████████████████████████████████| 2.4 MB 5.1 MB/s 


In [ ]:
#pip install pyprojroot

In [ ]:
#importamos librerias
import pyreadstat
import pandas as pd
from pyprojroot import here 
#from time import time, ctime
import datetime
#from janitor import clean_names
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Cargando la data
#data, meta = pyreadstat.read_sav(here() / "Input/Credito prendario.sav")
data, meta = pyreadstat.read_sav("Credito prendario.sav")

## **1. Transformación y limpieza de la BD**

In [ ]:
# Nos quedaremos con los registros del último mes
data=data[data.Periodo == data["Periodo"].max()] #max de periodo 201912

In [ ]:
# Nos quedaremos con los registros que no cayeron en adjudicación y por lo tanto se puede establecer alguna estrategia a tiempo
#data=data[data.FlagAdj == 0]

In [ ]:
#Eliminar los casos con Edad menor a 18 años:
data = data[data.Edad >= 18]

In [ ]:
#Eliminar los casos con desembolso=0:
data = data[data.Desem > 0]

In [ ]:
#Estableciendo un umbral al ratio de desembolso y tasación, en clase el profesor
#Chincaro nos indicó 0.99 para no eliminar mucha información
data['ratio_desem_tasa'] = data['Desem']/data['Monto_Tasación']
data = data[data.ratio_desem_tasa <= 0.99]

In [ ]:
#Editando variables de tiempo
data['t_Periodo'] = pd.to_datetime(data['Periodo'], format='%Y%m')

In [ ]:
#Eliminando variables de no interés
del data['ratio_desem_tasa'],data['FlagAtra'],data['Periodo'],data['FlagAdj']

In [ ]:
#Dicotomizando
#EstaCiv 1:CASADO, 2: CONVIVIENTE, 3: DIVORCIADO, 4: SEPARADO, 5: SOLTERO, 6: VIUDO
data['EstaCiv_cas'] = data['EstaCiv']
data['EstaCiv_con'] = data['EstaCiv']
data['EstaCiv_div'] = data['EstaCiv']
data['EstaCiv_sep'] = data['EstaCiv']
data['EstaCiv_sol'] = data['EstaCiv']
data.replace(to_replace={"EstaCiv_cas": {2:0,3:0,4:0,5:0,6:0},"EstaCiv_con": {1:0,2:1,3:0,4:0,5:0,6:0},"EstaCiv_div": {1:0,2:0,3:1,4:0,5:0,6:0},"EstaCiv_sep": {1:0,2:0,3:0,4:1,5:0,6:0},"EstaCiv_sol": {1:0,2:0,3:0,4:0,5:1,6:0}},inplace=True)

#TipoViv 1:ALQUILADA, 2: ALQUILER VENTA, 3: DE FAMILIA, 4: PROPIA
data['TipoViv_alq'] = data['TipoViv']
data['TipoViv_def'] = data['TipoViv']
data['TipoViv_pro'] = data['TipoViv']
data.replace(to_replace={"TipoViv_alq": {2:0,3:0,4:0},"TipoViv_def": {1:0,2:0,3:1,4:0},"TipoViv_pro": {1:0,2:0,3:0,4:1}},inplace=True)

#Eliminando variables de no interés
del data['EstaCiv'],data['TipoViv']

In [ ]:
#Se esta elimnando las variables de "Desem", 'Monto_Tasación' y "MaxGramOro" por ener una alta correlación
#data=data.drop(['Desem','Monto_Tasación','MaxGramOro'],axis=1)

In [ ]:
data=data.reset_index(drop=True)

## **2. Aplicando modelo LGBM**

In [ ]:
import requests
import pickle
from google.cloud import storage

In [ ]:
#storage_client = storage.Client()
#bucket = storage_client.get_bucket('model-credito-prendario')
#blob = bucket.blob('classifier.pickle')
#blob.download_to_filename('/tmp/classifier.pickle')
#model_pk = pickle.load(open('/tmp/classifier.pickle', 'rb'))
model_pk_MLS = pickle.load(open('model_lgbm_gridsearch.pickle', 'rb'))


In [ ]:
data_MLS=np.array(data[['Edad','Sexo','CantCred','SalVigSol','DiasEfec','MaxDeuda',
                          'MaxDiasAtra','EstaCiv_cas','EstaCiv_con','EstaCiv_div',
                          'EstaCiv_sep','EstaCiv_sol','TipoViv_alq','TipoViv_def','TipoViv_pro']])

In [ ]:
prediction = model_pk_MLS.predict(data_MLS)

In [ ]:
data = pd.concat([data,pd.DataFrame({'Adj_pred':prediction})],axis=1)

In [ ]:
data['Adj_pred'].value_counts()

0.0    1004
Name: Adj_pred, dtype: int64

## **3. Aplicando modelo GMM**

In [ ]:
model_pk_MLNS = pickle.load(open('model_gmm.pickle', 'rb'))

In [ ]:
data_MLNS=np.array(data[["Edad","Sexo","CantCred","Desem","SalVigSol","Monto_Tasación","DiasEfec",
                            "MaxDeuda","MaxDiasAtra","MaxGramOro","AntiDias","EstaCiv_cas","EstaCiv_con",
                            "EstaCiv_div","EstaCiv_sep","EstaCiv_sol","TipoViv_alq","TipoViv_def","TipoViv_pro"]]) # Modificar de acuerdo a resultados en el modelo, usar datos scalar

In [ ]:
cluster = model_pk_MLNS.predict(data_MLNS)

In [ ]:
data = pd.concat([data,pd.DataFrame({'Cluster_MLNS':cluster})],axis=1)

In [ ]:
# Nos quedaremos con los registros que no cayeron en adjudicación y por lo tanto se puede establecer alguna estrategia a tiempo
data=data[data.Adj_pred == 1]

In [ ]:
data=data.reset_index(drop=True)